# Исследовательская работа с базой данных PostgreSQL 



Работа от имени аналитика крупного сервиса для чтения книг по подписке. 
<br/> Задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 
<br/> Цель - формулировка ценностного предложения для нового продукта.

<br/> Структура базы данных: 
![Схема БД](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=1640&userId=&cache=v2)

## Загрузка и просмотр таблиц БД

In [1]:
# импорт небходимых библиотек
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# установка параметров
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохранение коннектора
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

con=engine.connect()

In [3]:
# загрузка и просмотр пяти строк таблицы books
query = '''
SELECT *
FROM books
LIMIT 5;
        '''

pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [4]:
# загрузка и просмотр пяти строк таблицы authors
query = '''
SELECT *
FROM authors
LIMIT 5;
         '''

pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [5]:
# загрузка и просмотр пяти строк таблицы publishers
query = '''
SELECT *
FROM publishers
LIMIT 5;
         '''

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

In [6]:
# загрузка и просмотр пяти строк таблицы ratings
query = '''
SELECT *
FROM ratings
LIMIT 5;
         '''

pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [7]:
# загрузка и просмотр пяти строк таблицы reviews
query = '''
SELECT *
FROM reviews
LIMIT 5;
         '''
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Задачи исследования, SQL-запросы

### Количество книг, вышедших после 1 января 2000 года

In [8]:
# количество книг
query = '''
SELECT COUNT(book_id) AS books_count
FROM books
WHERE CAST(publication_date AS TIMESTAMP) >= '2000-01-01';
         '''

pd.io.sql.read_sql(sql=text(query), con = con)

,books_count
0,821


С 1 января 2000 года была опубликована 821 книга.

### Количество обзоров и средняя оценка для каждой книги

In [9]:
# количество обзоров и средняя оценка
query = '''
SELECT books.book_id as books_id,
       books.title as books_title,
       COUNT(DISTINCT reviews.text) as review_cnt,
       ROUND(AVG(ratings.rating), 2) as rating_avg
FROM books

LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = reviews.book_id

GROUP BY books_id,
         books_title
ORDER BY review_cnt DESC;
          '''
pd.io.sql.read_sql(sql=text(query), con = con)

,books_id,books_title,review_cnt,rating_avg
0,948,Twilight (Twilight #1),7,3.66
1,963,Water for Elephants,6,3.98
2,734,The Glass Castle,6,4.21
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
4,695,The Curious Incident of the Dog in the Night-Time,6,4.08
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,NaN
996,808,The Natural Way to Draw,0,NaN
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,NaN
998,221,Essential Tales and Poems,0,NaN


Лидирующую позицию по количеству обзоров занимает книга под названием "Twilight" и среднюю оценку 3,7 из 5-ти. Далее остальные книги имеют рецензии в количестве 6 и менее, а оценки варьируются.


### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [10]:
# количество книг по издательствам и средняя оценка
query = '''
SELECT publishers.publisher_id, 
       publishers.publisher, 
       COUNT(books.book_id) as books_count
FROM publishers
JOIN books ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher_id
ORDER BY books_count DESC
LIMIT 1;
         ''' 
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,books_count
0,212,Penguin Books,42


Наибольшее количество книг выпущено издетельством Penguin Books. В общем количестве 42 шт.

### Автор с самой высокой средней оценкой книг с 50 и более оценками

In [15]:
# автор с самой высокой средней оценкой
query = ''' 
SELECT authors.author, 
       ROUND(AVG(ratings.rating), 2) AS rating_avg
FROM books
JOIN ratings ON books.book_id = ratings.book_id
JOIN authors ON books.author_id = authors.author_id
WHERE books.book_id IN (SELECT book_id 
                        FROM ratings 
                        GROUP BY book_id 
                        HAVING COUNT(*) >= 50)
GROUP BY authors.author
ORDER BY rating_avg DESC
LIMIT 1; 
        '''

pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.29


При учете книг с оценками 50 и более самую высокую среднюю оценку имеет автор J.K. Rowling и Mary GrandPré(иллюстратор).

### Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [12]:
# автор с самой высокой оценкой книг c 50-тью и более
query = '''
SELECT ROUND(AVG(second.reviews), 3) AS reviews_count_avg
FROM (SELECT username,
      COUNT(rating) AS ratings
      FROM ratings
      GROUP BY username
      HAVING COUNT(rating) > 48) AS first
JOIN (SELECT username,
      COUNT(text) AS reviews
      FROM reviews
      GROUP BY username) AS second ON first.username=second.username;
         '''
pd.io.sql.read_sql(sql=text(query), con = con)

,reviews_count_avg
0,24.0


От пользователей, которые поставили более 48 оценок среднее количество обзоров составляет 24 шт.

### Дополнительное задание

Выведите таблицу, которая будет содержать по году публикации:

- количество издательств,

- выпущенных книг 

- сколько всего тысяч страниц было в изданных книгах

(отобразить только те года, в которых издано более 30 книг)

In [13]:
query = ''' 
SELECT 
    EXTRACT(YEAR from (books.publication_date)) AS year,
    COUNT(DISTINCT publishers.publisher_id) AS publishers_count,
    COUNT(DISTINCT books.book_id) AS books_count,
    SUM(books.num_pages) / 1000 AS pages_of_1000
FROM books
JOIN publishers ON books.publisher_id=publishers.publisher_id
GROUP BY year
HAVING COUNT(DISTINCT books.book_id) > 30
ORDER BY year;
        '''
pd.io.sql.read_sql(sql=text(query), con = con)

,year,publishers_count,books_count,pages_of_1000
0,1999.0,26,41,15
1,2000.0,35,38,13
2,2001.0,41,60,21
3,2002.0,62,94,38
4,2003.0,65,105,41
5,2004.0,88,124,46
6,2005.0,89,139,55
7,2006.0,109,184,68
8,2007.0,38,50,18


В период с 1999 по 2007 в год издавалось более 30 книг.

## Выводы
В ходе исследования была проанализирована база данных сервиса чтения книг.

База данных включает 5 таблиц, схема которых представлена в описании задачи:
- books
- authors
- publishers
- ratings
- reviews.

Итоги по запросам:
1. Количество книг, вышедших после 1 января 2000 года: 821 шт.
2. Лидирующую позицию по количеству обзоров занимает книга под названием "Twilight" и среднюю оценку 3,7 из 5-ти. Далее остальные книги имеют рецензии в количестве 6 и менее, а оценки варьируются. В выведенной таблице еть данные по всем книгам.
3. Больше всего книг выпустило издательство Penguin Books
4. Автор с самой высокой средней оценкой книг: J.K. Rowling
5. Пользователи, оставившие более 50 оценок, в среднем написали 24 ревью.

***